In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("My Decks")

---

In [ ]:
import matplotlib.patches as mpatches
from matplotlib.colors import rgb_to_hsv, hex2color
from matplotlib.gridspec import GridSpec


def is_light_color(color):
    # Convert color to RGB if it's in hex format
    if isinstance(color, str):
        color = hex2color(color)
    # Convert RGB to HSV
    hsv = rgb_to_hsv(color)
    # Check the brightness (value in HSV)
    return hsv[2] > 0.6


def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct * total / 100.0))
        return f"{pct:.0f}%\n({val})"

    return my_autopct


def deck_composition(deck_df, spacing=(2, 1), size=(5, 5), ring=0.3, **kwargs):
    temp = deck_df.copy()
    temp["Primary type"] = deck_df["Primary type"].fillna(deck_df["Card type"])
    main_df = temp[temp["Section"] == "Main"].groupby(["Deck", "Primary type"])["Count"].sum().unstack(0)
    extra_df = temp[temp["Section"] == "Extra"].groupby(["Deck", "Primary type"])["Count"].sum().unstack(0)
    side_df = temp[temp["Section"] == "Side"].groupby(["Deck", "Primary type"])["Count"].sum().unstack(0)

    # Font sizes
    label_font_size = kwargs.get("label_font_size", 14)
    title_font_size = kwargs.get("title_font_size", 16)
    suptitle_font_size = kwargs.get("suptitle_font_size", 20)
    legend_font_size = kwargs.get("legend_font_size", 12)

    plot_width = size[0]  # Width of each plot
    plot_height = size[1]  # Fixed height for each plot
    horizontal_space = spacing[0]  # Fixed horizontal space between plots
    vertical_space = spacing[1]  # Fixed vertical space between plots
    suptitle_to_legend_space = kwargs.get("suptitle_to_legend_space", 0.5)  # Fixed space between suptitle and legend
    legend_to_first_row_space = kwargs.get(
        "legend_to_first_row_space", 1
    )  # Fixed space between legend and first row of plots

    decks = deck_df["Deck"].unique()
    cols = min(kwargs.get("min_cols", 3), len(decks))
    rows = int(np.ceil(len(decks) / cols))

    colors_main = [plot.colors_dict[type] for type in main_df.index]
    colors_extra = [plot.colors_dict[type] for type in extra_df.index]
    colors_remaining = side_df.index.difference(main_df.index.union(extra_df.index))

    # Dynamically calculate the figure size based on the number of rows and columns
    fig_width = plot_width * cols + (cols - 1) * horizontal_space
    fig_height = plot_height * rows + (rows - 1) * vertical_space + suptitle_to_legend_space + legend_to_first_row_space

    fig = plt.figure(figsize=(fig_width, fig_height))
    gs = GridSpec(
        nrows=rows,
        ncols=cols,
        wspace=horizontal_space / plot_width,  # Adjusted for figure width
        hspace=vertical_space / plot_height,  # Adjusted for plot height
    )
    fig.suptitle("Deck composition", fontsize=suptitle_font_size)

    for i, deck in enumerate(decks):
        # Create sub-grid for pie and bar plots
        sub_gs = gs[(i // cols), i % cols].subgridspec(2, 1, height_ratios=[9, 1], hspace=0.2)

        # Main plot in the upper sub-grid
        ax_pie = fig.add_subplot(sub_gs[0, 0])
        wedges1, texts1, autotexts1 = ax_pie.pie(
            main_df[deck].dropna(),
            autopct=make_autopct(main_df[deck].dropna()),
            startangle=90,
            radius=1,
            wedgeprops=dict(width=ring, edgecolor="w"),
            pctdistance=0.85,
            colors=np.array(colors_main)[main_df[deck].notna()],
            counterclock=False,
        )

        if deck in extra_df.columns:
            wedges2, texts2, autotexts2 = ax_pie.pie(
                extra_df[deck].dropna(),
                autopct=make_autopct(extra_df[deck].dropna()),
                startangle=90,
                radius=1 - ring,
                wedgeprops=dict(width=ring, edgecolor="w"),
                pctdistance=0.75,
                colors=np.array(colors_extra)[extra_df[deck].notna()],
                counterclock=False,
            )

        for wedge, text in zip(wedges1, autotexts1):
            color = wedge.get_facecolor()[:3]
            text.set_color("black" if is_light_color(color) else "white")
        for wedge, text in zip(wedges2, autotexts2):
            color = wedge.get_facecolor()[:3]
            text.set_color("black" if is_light_color(color) else "white")

        ax_pie.text(
            0,
            0,
            f"Main: {main_df[deck].sum()}\nExtra: {extra_df[deck].sum()}",
            ha="center",
            va="center",
            fontsize=label_font_size,
        )
        ax_pie.set_title(deck, fontsize=title_font_size)
        ax_pie.set_xlim(-1, 1)
        ax_pie.set_ylim(-1, 1)
        ax_pie.set_aspect("equal", adjustable="box")

        ax_bar = fig.add_subplot(sub_gs[1, 0])  # Bar plot in the odd row
        ax_bar.axis("off")
        # Create bar plot in the lower sub-grid
        if deck in side_df and side_df[deck] is not None:
            sorted_side = side_df[deck].sort_values(ascending=True).dropna()
            side_total = sorted_side.sum()
            left = 0
            height = 0.1
            for j, (name, count) in enumerate(sorted_side.items()):
                left -= count
                color = plot.colors_dict[name]
                bc = ax_bar.barh(
                    0,
                    width=count,
                    height=height,
                    left=left,
                    color=color,
                    edgecolor="white",
                )
                ax_bar.bar_label(
                    bc,
                    labels=[f"{count/side_total*100:.0f}%\n({count})"],
                    label_type="center",
                    color="black" if is_light_color(color) else "white",
                )
            ax_bar.set_title(f"Side: {side_total}", fontsize=label_font_size)
            ax_bar.set_xlim(-side_total, 0)
            ax_bar.set_ylim(-0.05, 0.05)
            ax_bar.set_aspect(side_total, adjustable="box")

        else:
            ax_bar.set_title(f"Side: 0", fontsize=label_font_size)

    # Create custom legend handles for main_df and extra_df
    colors_main += [
        plot.colors_dict[type]
        for type in colors_remaining
        if type not in ["Fusion Monster", "Synchro Monster", "Xyz Monster", "Link Monster"]
    ]
    colors_main += [
        plot.colors_dict[type]
        for type in colors_remaining
        if type in ["Fusion Monster", "Synchro Monster", "Xyz Monster", "Link Monster"]
    ]
    handles1 = [mpatches.Patch(color=plot.colors_dict[type], label=type) for type in main_df.index]
    handles2 = [mpatches.Patch(color=plot.colors_dict[type], label=type) for type in extra_df.index]

    # Adjust the legend position
    fig.legend(
        handles=handles1,
        title="Main deck",
        loc="upper center",
        fontsize=legend_font_size,
        ncol=len(handles1),
        bbox_to_anchor=(0.5, 1 - (suptitle_to_legend_space + legend_to_first_row_space + rows * 0.06 - 1) / fig_height),
        frameon=False,
        title_fontsize=legend_font_size + 2,
    )
    fig.legend(
        handles=handles2,
        title="Extra deck",
        loc="upper center",
        fontsize=legend_font_size,
        ncol=len(handles2),
        bbox_to_anchor=(0.5, 1 - (suptitle_to_legend_space + legend_to_first_row_space + rows * 0.06 - 0.5) / fig_height),
        frameon=False,
        title_fontsize=legend_font_size + 2,
    )

    fig.subplots_adjust(
        top=1 - (suptitle_to_legend_space + legend_to_first_row_space + rows / 2 * 0.1 + 0.6) / fig_height, bottom=0.1
    )

    return fig

In [ ]:
from matplotlib.gridspec import GridSpec


def deck_distribution(deck_df, column, spacing=(3, 1.2), size=None, colors=None, **kwargs):
    decks = deck_df[deck_df[column].notna()]["Deck"].unique()
    max_label = max([len(x) for x in deck_df[column].dropna().unique()])
    mean_labels = deck_df.groupby("Deck")[column].nunique()
    mean_labels = mean_labels[mean_labels > 0].mean()

    # Font sizes
    label_font_size = kwargs.get("label_font_size", 14)
    title_font_size = kwargs.get("title_font_size", 20)
    legend_font_size = kwargs.get("legend_font_size", 12)

    # Set constants for plot sizes and spacing
    plot_width = 6 if size is None else size[0]  # Width of each plot
    plot_height = max(mean_labels / 2, 0.5) if size is None else size[1]  # Fixed height for each plot
    horizontal_space = spacing[0] + max(int(max_label / 10 - 1), 0)  # Fixed horizontal space between plots
    vertical_space = spacing[1]  # Fixed vertical space between plots
    suptitle_to_legend_space = kwargs.get("suptitle_to_legend_space", 0.5)  # Fixed space between suptitle and legend
    legend_to_first_row_space = kwargs.get(
        "legend_to_first_row_space", 1
    )  # Fixed space between legend and first row of plots

    # Calculate number of columns and rows
    cols = min(kwargs.get("max_cols", 2), len(decks))
    rows = int(np.ceil(len(decks) / cols))

    # Dynamically calculate the figure size based on the number of rows and columns
    fig_width = plot_width * cols + (cols - 1) * horizontal_space
    fig_height = plot_height * rows + (rows - 1) * vertical_space + suptitle_to_legend_space + legend_to_first_row_space

    fig = plt.figure(figsize=(fig_width, fig_height))
    gs = GridSpec(
        nrows=rows,
        ncols=cols,
        wspace=horizontal_space / plot_width,  # Adjusted for figure width
        hspace=vertical_space / plot_height,  # Adjusted for plot height
    )

    if colors is None:
        colors = {
            section: plot.colors_dict[c]
            for section, c in zip(["Main", "Extra", "Side"], ["Effect Monster", "Fusion Monster", "Counter"])
        }
    else:
        colors = {
            section: colors[i] for i, section in enumerate(deck_df[deck_df[column].notna()]["Section"].dropna().unique())
        }

    # Plotting each deck's data
    temp = deck_df.copy()
    for i, deck in enumerate(decks):
        temp_df = temp[temp["Deck"] == deck].groupby(["Section", column])["Count"].sum().unstack(0)
        temp_df = temp_df[temp_df.sum().sort_values(ascending=False).index]
        color = [colors[section] for section in temp_df.columns]
        if not temp_df.empty:
            ax = fig.add_subplot(gs[i // cols, i % cols])
            temp_df.plot.barh(ax=ax, stacked=True, legend=False, fontsize=label_font_size, color=color)
            ax.set_ylabel("")
            ax.set_xlabel("Count", fontsize=label_font_size)
            ax.set_title(deck, fontsize=label_font_size)
            ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))

    # Adjust margins and add suptitle
    fig.suptitle(f"{column} distribution", fontsize=title_font_size)

    # Add legend with a fixed position
    fig.legend(
        [f"{section} deck" for section in sorted(temp[temp[column].notna()]["Section"].unique())],
        loc="lower center",
        fontsize=legend_font_size,
        ncol=3,
        bbox_to_anchor=(0.5, 1 - (suptitle_to_legend_space + legend_to_first_row_space + rows * 0.06 - 0.6) / fig_height),
        borderaxespad=0.5,
        frameon=False,
    )

    # Adjust subplot parameters to ensure fixed top and bottom margins
    fig.subplots_adjust(
        top=1 - (suptitle_to_legend_space + legend_to_first_row_space + rows / 2 * 0.1 - 0.3) / fig_height, bottom=0.1
    )

    return fig

# Data loading

In [ ]:
_ = git.ensure_repo()

## Read decks

In [ ]:
# Timestamp
timestamp = arrow.utcnow()

In [ ]:
# Load decks from YDK and decklist files
deck_df = pd.concat([get_ydk(), get_decklists()], ignore_index=True)

In [ ]:
# Process the deck data frame
deck_df = find_cards(deck_df, merge_data=True)

## Changelog

In [ ]:
# Get latest file if exist
previous_df, previous_ts = load_latest_data("deck")

if previous_df is not None:
    previous_df = previous_df.astype(deck_df[previous_df.columns.intersection(deck_df.columns)].dtypes.to_dict())

In [ ]:
if previous_df is None:
    deck_changelog = None
    print("Skipped")
else:
    deck_changelog = generate_changelog(previous_df, deck_df, col="Name")
    if not deck_changelog.empty:
        display(deck_changelog)
        deck_changelog.to_csv(
            dirs.DATA
            / make_filename(
                report="deck",
                timestamp=timestamp,
                previous_timestamp=previous_ts,
            ),
            index=True,
        )
        print("Changelog saved")

In [ ]:
if deck_changelog is not None and deck_changelog.empty:
    print("No changes. New data not saved")
else:
    deck_df.to_csv(
        dirs.DATA / make_filename(report="deck", timestamp=timestamp),
        index=False,
    )
    print("Data saved")

# Data visualization

In [ ]:
deck_df

## Check collection

In [ ]:
# Other

# Merge the collection and deck data frames
collection_df = get_collection()
if collection_df is not None:
    collection_df = assign_deck(collection_df, deck_df=deck_df, return_collection=False)

In [ ]:
collection_df

## Deck composition

In [ ]:
_ = deck_composition(deck_df)
plt.show()

## Secondary type

In [ ]:
_ = deck_distribution(deck_df.explode("Secondary type"), "Secondary type")
plt.show()

## Monster type

In [ ]:
_ = deck_distribution(deck_df, "Monster type")
plt.show()

## Properties

In [ ]:
_ = deck_distribution(deck_df, "Property")
plt.show()

# TCG & OCG Status 

In [ ]:
fig = deck_distribution(deck_df, "Property")
plt.show()